# Tasty Bytes - RAG Chatbot Using Cortex and Streamlit

In [ ]:
-- verify current environment
SELECT 
    CURRENT_WAREHOUSE(),
    CURRENT_DATABASE(),
    CURRENT_SCHEMA(),
    CURRENT_ROLE();

## Step 1 : TastyBytes Chatbot Database Setup

### Overview
This section establishes the core database infrastructure for the TastyBytes chatbot application. The setup includes:
- Main application database
- Dedicated application schema
- Compute warehouse configuration

### Architecture Components
1. **Database Layer** - tasty_bytes_chatbot database serves as the main container
2. **Schema Layer** - app schema organizes related objects
3. **Compute Layer** - Large warehouse handles processing needs

### Technical Implementation
The setup uses Snowflake's CREATE OR REPLACE syntax for idempotent deployment:
- Database creation establishes the top-level container
- Schema creation provides logical organization
- Warehouse configuration optimizes for performance and cost

In [ ]:
-- Database
CREATE OR REPLACE DATABASE tasty_bytes_chatbot;

--Schema
CREATE OR REPLACE SCHEMA tasty_bytes_chatbot.app;

--Warehouse
CREATE OR REPLACE WAREHOUSE tasty_bytes_chatbot_wh with
WAREHOUSE_SIZE = LARGE
AUTO_SUSPEND = 60;

## Step 2: Data Loading Infrastructure Setup

### Overview
This section configures essential components for loading data into the TastyBytes chatbot system:
- CSV file format specification
- External stage connection to S3 bucket

### Components
1. **File Format** - Defines CSV parsing rules
2. **External Stage** - Creates secure connection to S3 source data

### Technical Details
The setup enables automated data ingestion from the TastyBytes quickstart S3 bucket:
- File format supports CSV data processing
- S3 stage connects to pre-populated sample data
- Components are created in the app schema for organized access

In [ ]:
-- Create file format for CSV processing
CREATE OR REPLACE FILE FORMAT tasty_bytes_chatbot.app.csv_ff 
TYPE = 'csv';

-- Create external stage to connect to S3 data source
CREATE OR REPLACE STAGE tasty_bytes_chatbot.app.s3load
COMMENT = 'Quickstarts S3 Stage Connection'
url = 's3://sfquickstarts/tastybytes-cx/app/'
file_format = tasty_bytes_chatbot.app.csv_ff;

## Step 3 : Document Storage Setup

### Overview
This section establishes the document storage infrastructure for the TastyBytes chatbot:
- Creates a table for storing document metadata and content
- Loads initial document data from S3 stage

### Table Structure
The documents table contains:
- RELATIVE_PATH: Document location identifier
- RAW_TEXT: Actual document content
Both fields use VARCHAR with maximum capacity for flexibility

### Implementation Notes
The setup includes:
- Table creation with JSON metadata for tracking
- Bulk data load from staged S3 files
- Integration with previously configured external stage

In [ ]:
-- Create table for storing document content and metadata
CREATE OR REPLACE TABLE tasty_bytes_chatbot.app.documents (
	RELATIVE_PATH VARCHAR(16777216),
	RAW_TEXT VARCHAR(16777216)
)
COMMENT = '{"origin":"sf_sit-is", "name":"voc", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":1, "source":"streamlit", "vignette":"rag_chatbot"}}';

-- Load initial document data from S3 stage
COPY INTO tasty_bytes_chatbot.app.documents
FROM @tasty_bytes_chatbot.app.s3load/documents/;

## Step 4 : Vector Store Implementation

### Overview
This section implements the vector storage system for embeddings:
- Creates temporary array-based staging table
- Creates final vector store with proper VECTOR data type
- Transforms array data into optimized vector format

### Architecture
The implementation follows a two-step process:
1. **Staging Layer** - array_table stores raw array data
2. **Production Layer** - vector_store contains optimized VECTOR columns

### Technical Details
- Uses ARRAY type for initial data loading (Snowflake requirement)
- Converts to VECTOR(FLOAT, 768) for optimized storage and querying
- Maintains source tracking and text content alongside embeddings
- VECTOR type cannot be loaded directly, requiring ARRAY intermediate storage
- 768-dimension vectors match standard NLP embedding models
- VARCHAR fields use maximum length for flexibility
- Source tracking enables multi-source embedding management



In [ ]:
-- https://docs.snowflake.com/en/sql-reference/data-types-vector#loading-and-unloading-vector-data
-- Create staging table with ARRAY type for initial data load
CREATE OR REPLACE TABLE tasty_bytes_chatbot.app.array_table (
  SOURCE VARCHAR(6),
	SOURCE_DESC VARCHAR(16777216),
	FULL_TEXT VARCHAR(16777216),
	SIZE NUMBER(18,0),
	CHUNK VARCHAR(16777216),
	INPUT_TEXT VARCHAR(16777216),
	CHUNK_EMBEDDING ARRAY
);

-- Load vector data from S3 into staging table
COPY INTO tasty_bytes_chatbot.app.array_table
FROM @tasty_bytes_chatbot.app.s3load/vector_store/;

-- Create optimized vector store with proper VECTOR type
CREATE OR REPLACE TABLE tasty_bytes_chatbot.app.vector_store (
	SOURCE VARCHAR(6),
	SOURCE_DESC VARCHAR(16777216),
	FULL_TEXT VARCHAR(16777216),
	SIZE NUMBER(18,0),
	CHUNK VARCHAR(16777216),
	INPUT_TEXT VARCHAR(16777216),
	CHUNK_EMBEDDING VECTOR(FLOAT, 768)
) AS
SELECT 
  source,
	source_desc,
	full_text,
	size,
	chunk,
	input_text,
  chunk_embedding::VECTOR(FLOAT, 768)
FROM tasty_bytes_chatbot.app.array_table;

##  Step 5 : Create Streamlit App

## TastyBytes Support Chatbot Implementation

### Overview
This Streamlit application implements a RAG-based customer support chatbot that:
- Uses Cortex LLMs for generating responses
- Leverages vector similarity search for relevant context
- Maintains chat history for contextual responses
- Allows model selection flexibility

### Architecture Components
1. **Core Dependencies**
   - Snowpark for data access
   - Cortex for LLM operations
   - Streamlit for UI/UX
   - Pandas for data manipulation

2. **Key Features**
   - Model selection from multiple LLM options
   - Vector similarity-based document retrieval
   - Context-aware response generation
   - Chat history management

### Implementation Notes
- Uses VECTOR_COSINE_SIMILARITY for document matching
- Maintains 20-message chat history
- Sanitizes inputs for SQL safety
- Provides real-time status updates

### Configuration Steps
1. **Access Streamlit**
   - Open Snowsight interface
   - Navigate to "Projects" menu
   - Select "Streamlit" tab

2. **Create New Application**
   - Click "+ Streamlit App" button
   - Enter application name
   - Configure required resources:
     * Database: TASTY_BYTES_CHATBOT
     * Schema: APP
     * Warehouse: TASTY_BYTES_CHATBOT_WH

3. **Setup Dependencies**
   - Open code editor section
   - Access "Packages" dropdown menu
   - Add snowpark-ml-python package
   - This enables ML capabilities for the chatbot

4. **Deploy Application**
   - Copy implementation code to editor
   - Click "Run" to deploy application
   - Monitor deployment status

### Technical Requirements
- Active Snowflake account with appropriate privileges
- Access to specified database and schema
- Configured warehouse with adequate resources
- Required Python packages available

# Cleanup

In [ ]:
-- Drop tables
DROP TABLE IF EXISTS tasty_bytes_chatbot.app.vector_store;
DROP TABLE IF EXISTS tasty_bytes_chatbot.app.array_table;
DROP TABLE IF EXISTS tasty_bytes_chatbot.app.documents;

In [ ]:
use warehouse compute_wh;
use database notebooks_db;

In [ ]:
SELECT 
    CURRENT_WAREHOUSE(),
    CURRENT_DATABASE(),
    CURRENT_SCHEMA(),
    CURRENT_ROLE();

In [ ]:


-- Drop stage
DROP STAGE IF EXISTS tasty_bytes_chatbot.app.s3load;

-- Drop file format
DROP FILE FORMAT IF EXISTS tasty_bytes_chatbot.app.csv_ff;

-- Drop schema
DROP SCHEMA IF EXISTS tasty_bytes_chatbot.app;

-- Drop warehouse
DROP WAREHOUSE IF EXISTS tasty_bytes_chatbot_wh;

-- Drop database
DROP DATABASE IF EXISTS tasty_bytes_chatbot;